In [1]:
import numpy as np
import cv2
import multiprocessing
import matplotlib.pyplot as plt
import time
from my_args import MyArgs
from video_extracter import VideoExtracter
import pickle

In [2]:
def get_output_dim(size, h, w):
        if isinstance(size, tuple) and len(size) == 2:
            return size
        elif h >= w:
            return int(h * size / w), size
        else:
            return size, int(w *size / h)

In [3]:
args = MyArgs(model_type='3d',batch_size = 10,framerate=30,scale_time=2)

In [4]:
v_extracter = VideoExtracter(args)

Loading 3D-ResneXt-101 ...
loaded


***

In [5]:
def start_camera(model_type,time_split,v_extracter_,clf_):
    cap = cv2.VideoCapture(0)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    weight = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h, w = get_output_dim(112,height,weight)
    delays = []
    video = []    
    warningflag = False
    # while (countstop < frame_rate*5):
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()        
        # Our operations on the frame come here
        if model_type == '3d':
            vframe = cv2.resize(frame[:,::-1,:],(112,112),fx=0,fy=0, interpolation = cv2.INTER_AREA)
        elif model_type == '2d':
            vframe = cv2.resize(frame[:,::-1,:],(w,h),fx=0,fy=0, interpolation = cv2.INTER_AREA)
        else:
            print('Model type must be: 2d or 3d')
            break        
        video.append(vframe)
        if len(video) == frame_rate * time_split:
            temp = video.copy()
            video.clear()
            start = time.time()
            feature = v_extracter_.predict(temp)            
            pred = clf_.predict(feature.flatten().reshape(1,-1))
            if pred == 1:
                warningflag = True
            else:
                warningflag = False
            end = time.time()
            delays.append(end - start)
        img = np.array(frame[:,::-1,:])        
        if warningflag:
            img = cv2.putText(img, 'Warnning: fighting detected', (20,30), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0,0,255), 1, cv2.LINE_AA)
        # Display the resulting frame
        cv2.imshow('Adaboost',img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    return delays

In [6]:
# You change model's absolute path to your path
clf_ada = pickle.load(open('./model_ml/model/01_adaboost.sav', 'rb'))

In [1]:
delays = start_camera('3d',args.scale_time,v_extracter,clf_ada)

In [ ]:
np.mean(delays)